## Про идентификацию имён

<!---В первую очередь стоит отметить, что для корректности получаемых результатов было бы логичнее минимизировать ошибки второго рода, поскольку они могут быть исправлены вручную. -->

В текущей реализации словаря ```natasha``` или ```mystem``` не использованы, поскольку они могут порождать дополнительные ошибки, а также очень тяжеловесны в плане временных затрат. Единственный способ использования подобных инструментов видится в выделении отдельных элементов ФИО и их нормализация, но ввиду того что данные ожидаются нормализованными в этом нет большой необходиомсти.

Пока единственный вариант применения видится как проверка инициалов, где с помощью ```natasha``` извлекаются ИО и далее по словарю, который учитывает тип слова проверяется. Проверку желательно фамилий нужно будет проводить отдельно.  

In [1]:
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsNERTagger, PER, NamesExtractor, Doc)

def natasha_split_name(name : str) -> dict:
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    ner_tagger = NewsNERTagger(emb)
    morph_vocab = MorphVocab()
    names_extractor = NamesExtractor(MorphVocab())
    doc = Doc(name)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    for span in doc.spans:
        if span.type == PER:
            span.extract_fact(names_extractor)
    return {_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}

In [2]:
from pymystem3 import Mystem

text = 'Николай Андреевич Римский-Корсаков'
def mystem_split_name(text : str) -> dict:
    m = Mystem()

    analyze = m.analyze(text)

    first_name = None
    second_name = None
    middle_name = None

    for word in analyze:
        try:
            analysis = word['analysis'][0]
        except KeyError:
            continue

        if 'имя' in analysis['gr']:
            first_name = word['text'].capitalize()
        elif 'фам' in analysis['gr']:
            second_name = word['text'].capitalize()
        elif 'отч' in analysis['gr']:
            middle_name = word['text'].capitalize()
    return {"first" : first_name, "middle" : middle_name, "last" : second_name}


In [3]:
print("natasha" + str(natasha_split_name("Петров И.")))
print("mystem" + str(mystem_split_name("Петров И.")))


natasha{'Петров И.': {'first': 'И', 'last': 'Петров'}}
mystem{'first': None, 'middle': None, 'last': 'Петров'}


Сравнение ```natasha``` или ```mystem``` на сложных моментах.

In [4]:
print("mystem" + str(mystem_split_name("Хармс АВ")))
print("natasha" + str(natasha_split_name("Хармс АВ")))
print("mystem" + str(mystem_split_name("Хармс А.В.")))
print("natasha" + str(natasha_split_name("Хармс А.В.")))
print("mystem" + str(mystem_split_name("Николай Андреевич Римский-Корсаков")))
print("natasha" + str(natasha_split_name("Николай Андреевич Римский-Корсаков")))
print("mystem" + str(mystem_split_name("Кузьма Сергеевич Петров-Водкин")))
print("natasha" + str(natasha_split_name("Кузьма Сергеевич Петров-Водкин")))
print("mystem" + str(mystem_split_name("Красавцев Иван Губинич")))
print("natasha" + str(natasha_split_name("Красавцев Иван Губинич")))
print("mystem" + str(mystem_split_name("Орлов Иван Губинич")))
print("natasha" + str(natasha_split_name("Орлов Иван Губинич")))
print("mystem" + str(mystem_split_name("Ева-София Пономарева")))
print("natasha" + str(natasha_split_name("Ева-София Пономарева")))

mystem{'first': 'Ав', 'middle': None, 'last': 'Хармс'}
natasha{}
mystem{'first': None, 'middle': None, 'last': 'Хармс'}
natasha{'Хармс А.В.': {'first': 'А', 'last': 'Хармс', 'middle': 'В'}}
mystem{'first': 'Николай', 'middle': 'Андреевич', 'last': 'Корсаков'}
natasha{'Николай Андреевич Римский-Корсаков': {'first': 'Николай', 'last': 'Римский', 'middle': 'Андреевич'}}
mystem{'first': 'Кузьма', 'middle': 'Сергеевич', 'last': 'Водкин'}
natasha{'Кузьма Сергеевич Петров-Водкин': {'first': 'Кузьма', 'last': 'Петров', 'middle': 'Сергеевич'}}
mystem{'first': 'Иван', 'middle': 'Губинич', 'last': None}
natasha{'Иван Губинич': {'first': 'Иван', 'last': 'Губинич'}}
mystem{'first': 'Иван', 'middle': 'Губинич', 'last': 'Орлов'}
natasha{'Орлов Иван Губинич': {'first': 'Иван', 'last': 'Орлов'}}
mystem{'first': 'Ева', 'middle': None, 'last': 'Пономарева'}
natasha{'Ева-София Пономарев': {'first': 'Ева'}}


Примеры работы с текущей реализацией словаря

При создании словаря все полные имена читаются из файла, и разбиваются на токены, которые преобразуются во множества.

При обработке сначала строка аналогичным образом разбивается на токены и проверяется, что множество токенов строки является подмножеством токенов имени из словаря. Если это так, то имя добавляется в возвращаемый список. Если в результате перебора словаря имя не нашлось, то запускается проверка на инициалы. Обрабатываются два вида скоращений: Фамилия И.О. и Фамилия И., сокращения могут располагаться как до, так и после фамилии. 

In [1]:
from common.names_dict import NamesDict
import nest_asyncio
nest_asyncio.apply()
x = NamesDict("resources\\names.xlsx")

print(x.get_names("Александер Хлебников"))
print(x.get_names("А.Б. Хармс"))
print(x.get_names("Манн Юля"))
print(x.get_names("Московская Женя"))
print(x.get_names("Витязев АВ"))
print(x.get_names("Хармс А.Б."))
print(x.get_names("Хармс АБ"))
print(x.get_names("Ахматова"))
print(x.get_names("Екатерина"))
print(x.get_names("Ахматова Екатерина Вадимовна"))
print(x.get_names("Ахматова Е."))
print(x.get_names("Белая Г."))

['Хлебников Александр']
['Хармс Александр Борисович']
['Юлия Манн']
['Московская Евгения']
['Витязев Андрей Викторович']
['Хармс Александр Борисович']
['Хармс Александр Борисович']
['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна', 'Грязнева Екатерина', 'Гумилева Екатерина Николаевна', 'Пирогова Екатерина', 'Щедрина Екатерина']
['Ахматова Екатерина Вадимовна']
['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
['Белая Галина']


## Построение графиков

Собираем информацию из таблички, в результате обработки выводится пара чисел (необработанные имена, все имена) 

In [1]:
from data_collector import DataCollector
import nest_asyncio
nest_asyncio.apply()

data_path = "data.xlsx"
structure_path = "resources\\survey_structure.json"
names_path = "resources\\names.xlsx"

collector = DataCollector(data_path, structure_path, names_path)
collector.collect()

row 117: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
row 192: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']


(2, 373)

Создаём отчётную таблицу

In [2]:
collector.create_report_df().to_excel("outputs\\report.xlsx", index = False)

c:\Users\zalex\AI-analysis-of-free-survey-responses\data_collector.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(data=[cur_row], columns = columns_names)], ignore_index = True)


Создаём отчёт по отдельному человеку и типу руководства

In [ ]:
collector.get_person_report("Илюхина Мария", "Непосредственный руководитель")

Помощь в решении проблемных вопросов_positive_pct                                                                                                 1.0
Помощь в решении проблемных вопросов_all                                                                                                            3
Эмоциональный настрой от совместной работы_positive_pct                                                                                           1.0
Эмоциональный настрой от совместной работы_all                                                                                                      3
Экспертность непосредственного руководителя_avg                                                                                              9.333333
Экспертность непосредственного руководителя_count                                                                                                   3
Вклад в улучшение и оптимизацию процессов подразделения_positive_pct                                

Определяем функцию, которая будет строить графики по для отдельных людей по выбранному типу руководства

In [ ]:
import plotly.graph_objects as go

def create_plot(name : str, group : str, collector : DataCollector):
    person_stat = collector.get_select_vals_for_plot(name, group)
    colors = ['rgba(168,209,141,0.8)',
            'rgba(251,114,134,0.8)']

    x_data = person_stat[1]
    y_data = person_stat[0]
    fig = go.Figure()

    for i in range(0, len(x_data[0])):
        for xd, yd in zip(x_data, y_data):
            fig.add_trace(go.Bar(
                x=[xd[i]], y=[yd],
                orientation='h',
                marker=dict(
                    color=colors[i],
                    line=dict(width=0)
                )
            ))

    fig.update_layout(
        xaxis=dict(
            showgrid=True,
            showline=False,
            tickformat= '0.00%',
            showticklabels=True,
            color="white",
            zeroline=True,
            domain=[0.15, 1]
        ),
        yaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
        ),
        barmode='stack',
        paper_bgcolor='rgb(0, 0, 0)',
        plot_bgcolor='rgb(0, 0, 0)',
        margin=dict(l=300, r=10, t=140, b=80),
        showlegend=False,
    )

    annotations = []

    for yd, xd in zip(y_data, x_data):
        # labeling the y-axis
        annotations.append(dict(xref='paper', yref='y',
                                x=0.14, y=yd,
                                xanchor='right',
                                text=str(yd),
                                font=dict(family='Arial', size=14,
                                        color='rgb(255, 255, 255)'),
                                showarrow=False, align='right'))
        # labeling the first percentage of each bar (x_axis)
        if xd[0]!=0:
            annotations.append(dict(xref='x', yref='y',
                                    x=xd[0] / 2, y=yd,
                                    text=str(int(xd[0]*100)) + '%',
                                    font=dict(family='Arial', size=14,
                                            color='rgb(0, 0, 0)'),
                                    showarrow=False))
        # labeling the second percentage of each bar (x_axis)
        if xd[1]!=0:
            annotations.append(dict(xref='x', yref='y',
                                    x=xd[0] + (xd[1]/2), y=yd,
                                    text=str(int(xd[1]*100)) + '%',
                                    font=dict(family='Arial', size=14,
                                                color='rgb(0, 0, 0)'),
                                    showarrow=False))

    fig.update_layout(annotations=annotations, title=dict(text=group, x=0.5 ,font=dict(size=50, color='rgb(255, 255, 255)')))
    fig.update_traces(width=0.5)
    fig.show()

По очереди строим примеры для непосредственных, вышестоящих и функциональных руководителей

In [10]:

create_plot("Губин Павел","Непосредственный руководитель", collector)

In [11]:
create_plot("Пушкин Иванович Валерий", "Вышестоящий руководитель", collector)

In [12]:
create_plot("Александр Альфонс", "Функциональный руководитель", collector)

Общая оценка экспертности по компании

In [13]:
print(collector.get_average_rating(4))
print(collector.get_average_rating([4]))
print(collector.get_average_rating([4,11]))
print(collector.get_average_rating([4,11,20]))

8.45374449339207
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Overall': 8.45374449339207}
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Экспертность вышестоящего руководителя': 8.384615384615385, 'Overall': 8.433962264150944}
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Экспертность вышестоящего руководителя': 8.384615384615385, 'Экспертность функционального руководителя': 7.5, 'Overall': 8.338983050847459}


In [ ]:
from pptx import Presentation
from pptx_utilities import select_slides, update_cards, update_text, update_table_cell, update_chart, create_slides_three_levels

def create_slides_l1(name: str, collector: DataCollector):
    pres = Presentation("resources\\Шаблон отчета.pptx")
    default_msg = "Недостаточно данных"
    select_slides(pres, [3])
    report = collector.get_person_report(name, "Непосредственный руководитель")

    #set positive/negative sides
    update_cards(pres.slides[0], report)

    #set name
    update_text(pres.slides[0].shapes[1].text_frame, name)

    #set feedback
    feedback = "Обратная связь:\n" + report["Обратная связь, непосредственный руководитель_feedback"]
    feedback = default_msg if report["Обратная связь, непосредственный руководитель_count"] < 2 else feedback
    update_text(pres.slides[0].shapes[3].text_frame, feedback)

    #set ratings
    ratings = collector.get_person_ratings(name)
    rating = default_msg if ratings[0][1] < 2 else f"{ratings[0][0]:.1f}"
    update_table_cell(pres.slides[0].shapes[2].table, 1, 1, rating)
    overall_rating = collector.get_average_rating([4,11,20])["Overall"]
    update_table_cell(pres.slides[0].shapes[2].table, 1, 2, f"{overall_rating:.2f}")
    
    pres.save(f"outputs\\{name}.pptx")

def create_slides_l123(name: str, collector: DataCollector):
    pres = Presentation("resources\\Шаблон отчета.pptx")
    default_msg = "Недостаточно данных"
    select_slides(pres, [1,2])
    report = collector.get_person_report(name)

    #update charts
    update_chart(pres.slides[0].shapes[3].chart, collector.get_select_vals_for_plot(name, "Непосредственный руководитель"))
    update_chart(pres.slides[1].shapes[1].chart, collector.get_select_vals_for_plot(name, "Вышестоящий руководитель"))
    update_chart(pres.slides[1].shapes[2].chart, collector.get_select_vals_for_plot(name, "Функциональный руководитель"))

    #set name
    update_text(pres.slides[0].shapes[1].text_frame, name)
    update_text(pres.slides[1].shapes[5].text_frame, name)

    #set feedback
    feedback = default_msg if report["Обратная связь, непосредственный руководитель_count"] < 2 else "Обратная связь:\n" + report["Обратная связь, непосредственный руководитель_feedback"]
    update_text(pres.slides[0].shapes[4].text_frame, feedback)

    feedback = default_msg if report["Обратная связь, вышестоящий руководитель_count"] < 2 else "Обратная связь:\n" + report["Обратная связь, вышестоящий руководитель_feedback"]
    update_text(pres.slides[1].shapes[3].text_frame, feedback)

    feedback = default_msg if report["Обратная связь, функциональный руководитель_count"] < 2 else "Обратная связь:\n" + report["Обратная связь, функциональный руководитель_feedback"]
    update_text(pres.slides[1].shapes[4].text_frame, feedback)

    #set ratings
    ratings = collector.get_person_ratings(name)
    rating = default_msg if ratings[0][1] < 2 else f"{ratings[0][0]:.1f}"
    update_table_cell(pres.slides[0].shapes[2].table, 1, 1, rating)

    rating = default_msg if ratings[1][1] < 2 else f"{ratings[1][0]:.1f}"
    update_table_cell(pres.slides[0].shapes[2].table, 1, 2, rating)

    rating = default_msg if ratings[2][1] < 2 else f"{ratings[2][0]:.1f}"
    update_table_cell(pres.slides[0].shapes[2].table, 1, 3, rating)

    overall_rating = collector.get_average_rating([4,11,20])["Overall"]
    update_table_cell(pres.slides[0].shapes[2].table, 1, 4, f"{overall_rating:.2f}")
    
    pres.save(f"outputs\\{name}.pptx")

def shapes_info(slide):
    for i, shape in enumerate(slide.shapes, 0):
        if shape.has_text_frame:
            print(f"{i}: text")
            print(f"\"{shape.text_frame.text}\"")
        if shape.has_table:
            print(f"{i}: table")
        if shape.has_chart:
            print(f"{i}: chart")

presentation = Presentation("resources\\Шаблон отчета.pptx")
shapes_info(presentation.slides[2])
#create_slides_one_level("Илюхина Мария", collector)
create_slides_three_levels("Губин Иван", collector)



0: text
"Оценка руководителей"
1: chart
2: chart
3: text
"Обратная связь"
4: text
"Обратная связь"
5: text
"ФИО руководителя подразделения"


In [2]:
from common.generate_text import async_get_name
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsNERTagger, PER, NamesExtractor, Doc
import nest_asyncio
nest_asyncio.apply()

name = "Манн Таня"
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)
morph_vocab = MorphVocab()
names_extractor = NamesExtractor(MorphVocab())
doc = Doc(name)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for token in doc.tokens:
    token.lemmatize(morph_vocab)
doc.tag_ner(ner_tagger)
for span in doc.spans:
    span.normalize(morph_vocab)
for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)
x = {"x": _.fact.as_dict for _ in doc.spans if _.type == PER}
print(x)
async_get_name(x["x"]["first"])

{'x': {'first': 'Таня', 'last': 'Манн'}}


'Татьяна'

In [3]:
from data_collector import DataCollector
from pptx import Presentation
from pptx_utilities import create_slides_one_level, create_slides_three_levels, create_slides_two_levels

def select_filled_groups(name : str, collector : DataCollector) -> frozenset[int]:
    structure = collector.survey_structure
    valid_groups = list()
    index = 1
    for group_name, columns in structure["groups"].items():
        groups = structure.create_group_structure(columns)
        counter  = 0 
        report = collector.get_person_report(name, group_name)
        for question_nmb in groups["select"]:
            if question_nmb in structure["output headers"] and report[structure["output headers"][question_nmb] + "_count"] > 0:
                counter += 1
                continue
            if question_nmb not in structure["output headers"] and report[structure["questions"][question_nmb][1] + "_count"] > 0:
                counter += 1
        if counter == len(groups["select"]):
            valid_groups.append(index)
        index += 1
    return valid_groups

def create_slides(name : str, collector : DataCollector):
    report = collector.get_person_report(name)
    if report is None:
        print(f"Name \"{name}\" not found")
        return
    
    filled_groups = select_filled_groups(name, collector)
    if len(filled_groups) == 0:
        print(f"For name \"{name}\" not enough data")
    elif len(filled_groups) == 1:
        create_slides_one_level(name, filled_groups[0], collector)
    elif len(filled_groups) == 2:
        create_slides_two_levels(name, tuple(filled_groups), collector)
    elif len(filled_groups) == 3:
        create_slides_three_levels(name, filled_groups[0], collector)
        return
    

#Махонин Александр
#Твен Василий
#Пушкин Валерий
#Диккенс Георгий
#print(select_filled_groups("Пушкин Валерий Иванович", collector))
#print(collector.get_person_report("Пушкин Валерий Иванович"))
create_slides("Пушкин Валерий Иванович", collector)

In [3]:
from pptx_utilities import print_shapes_info
from pptx import Presentation
pres = Presentation("resources\\template.pptx")

for i, slide in enumerate(pres.slides, 0):
    print(f"Slide {i}")
    print_shapes_info(slide)

Slide 0
0: text
"Название подразделения
Итоги 2024: вовлеченность, лояльность и оценка руководства"
1: text
""
Slide 1
0: text
"Оценка руководителей"
1: text
"ФИО руководителя подразделения"
2: table
3: chart
4: text
"Обратная cвязь"
Slide 2
0: text
"Оценка руководителей"
1: chart
2: chart
3: text
"Обратная связь"
4: text
"Обратная связь"
5: text
"ФИО руководителя подразделения"
Slide 3
0: text
"Оценка руководителей"
1: text
"ФИО руководителя подразделения"
2: table
3: chart
4: text
"Обратная cвязь"
Slide 4
0: text
"Оценка руководителей"
1: text
"ФИО руководителя подразделения"
2: table
3: chart
4: text
"Обратная cвязь"
Slide 5
0: text
"Оценка руководителей"
1: text
"ФИО руководителя линейного"
2: table
3: text
"Обратная связь"
4: text
"p0"
5: table
6: text
"n0"
7: text
"p1"
8: text
"p2"
9: text
"n1"
10: text
"n2"
11: text
"p3"
12: text
"n3"
13: text
"p4"
14: text
"n4"


In [3]:
res = collector.get_areas_of_growth()
for i, j  in res.items():
    print(i)
    print(j)
    print()

Непосредственный руководитель
                                            question    rating  votes
0                   Способствование развитию команды  0.775330    227
1              Процесс предоставления обратной связи  0.801762    227
2  Вклад в улучшение и оптимизацию процессов подр...  0.850220    227
3         Эмоциональный настрой от совместной работы  0.863436    227
4               Помощь в решении проблемных вопросов  0.872247    227

Вышестоящий руководитель
                                            question    rating  votes
0  Предоставление информации по ключевым изменени...  0.657143    105
1                               Эмоциональный вклад   0.714286    105
2  Вклад в улучшение и оптимизацию процессов подр...  0.761905    105

Функциональный руководитель
                                            question    rating  votes
0  Предоставление информации по ключевым изминени...  0.717949     39
1            Объединение кроссфункциональной команды  0.794872     39
2    

In [ ]:
collector.get_top_one_level(threshold = 1, top = 100)

,name,rating
0,Шершнев Павел,10.000000
1,Грязнева Екатерина,10.000000
2,Блажник Алексей Евгеньевич,10.000000
3,Критник Евгений,10.000000
4,Московская Евгения,10.000000
...,...,...
56,Инокеньтев Иван,4.571429
57,Тэффи Анастасия,4.000000
58,Комаров Евгений,3.000000
59,Гумилева Нина Александровна,2.000000


In [3]:
res = collector.get_top_several_levels(threshold = 1, top = 5)
print(res["direct"])
print(res["non direct"])

                 name  rating direct  rating non direct
0    Жуковский Кирилл           10.0          10.000000
1    Горбанидце Иосиф           10.0           8.000000
2  Колокольцев Виктор           10.0          10.000000
3          Лем Виктор           10.0          10.000000
4         Гарсиа Юрий           10.0           9.666667
                               name  rating direct  rating non direct
0  Ахматова Екатерина Александровна       9.750000               10.0
1                       Губин Павел       7.666667               10.0
2       Брагин Владимирович Дмитрий       9.000000               10.0
3                   Ведищева Амелия       8.000000               10.0
4                  Жуковский Кирилл      10.000000               10.0


In [ ]:
from pptx_collector import PptxCollector
import nest_asyncio
nest_asyncio.apply()

data_path = "data.xlsx"
structure_path = "resources\\survey_structure.json"
names_path = "resources\\names.xlsx"
pptx_template_path = "resources\\template.pptx"

collector = PptxCollector(data_path, structure_path, names_path)
collector.collect()
collector.create_slides("Илюхина", pptx_template_path)
collector.create_slides("Пушкин", pptx_template_path)
collector.create_slides("Витязев", pptx_template_path)
collector.create_slides("диккенс", pptx_template_path)



row 117: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
row 192: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
Name "None" not found
